In [1]:
## todo:
# оформить в единый код
# единая концепция: есть регрессоры, есть формулы смешивания, есть показатели, есть разные формулы, все np массивы

In [2]:
import pandas as pd # для работы с датасетами
import numpy as np # для массивов нампай

### Загружаем исходные массивы

In [3]:
# N = pd.read_csv("PAINeT train data-24-01-2023 N.csv",sep=';',decimal=',')
Odata = pd.read_csv("datasets/PAINeT train data-24-01-2023 O.csv",sep=';',decimal=',')
# N2 = pd.read_csv("PAINeT train data-24-01-2023 N2.csv",sep=';',decimal=',')
O2data = pd.read_csv("datasets/PAINeT train data-24-01-2023 O2.csv",sep=';',decimal=',')
# NO = pd.read_csv("PAINeT train data-24-01-2023 NO.csv",sep=';',decimal=',')

mixture = pd.read_csv("datasets/PAINeT train data-25-01-2023 N2-N-O2-O-NO.csv",sep=';',decimal=',')

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/PAINeT train data-24-01-2023 O.csv'

In [ ]:
species_type = {
    'N': True,
    'O': True,
    'N2': False,
    'O2': False,
    'NO': False
}

In [16]:
molecular_weights = {
    'N': 14.0067,
    'O': 15.999,
    'N2': 28.0134,
    'O2': 31.9988,
    'NO': 30.006
}

In [17]:
sutherland_constants = { ### double check this!!
    'O2': {'As': 1.458e-6, 'Ts': 120.0},
    'N2': {'As': 1.396e-6, 'Ts': 111.0},
    'NO': {'As': 1.368e-6, 'Ts': 97.0},
    'N': {'As': 1.434e-6, 'Ts': 91.0},
    'O': {'As': 1.584e-6, 'Ts': 106.0}
}


In [18]:
blottner_constants = {
    'NO': {
        'A': 0.0436378,
        'B': -0.0335511,
        'C': -9.5767430
    },
    'N': {
        'A': 0.0115572,
        'B': 0.6031679,
        'C': -12.4327495
    },
    'O': {
        'A': 0.0203144,
        'B': 0.4294404,
        'C': -11.6031403
    },
    'N2': {
        'A': 0.0268142,
        'B': 0.3177838,
        'C': -11.3155513
    },
    'O2': {
        'A': 0.0449290,
        'B': -0.0826158,
        'C': -9.2019475
    }
}


In [19]:
GisThetaDict = {
    "N" : [
        (4,  0),
        (10, 2.766470e4 ),
        (6,  4.149309e4)
    ],
    "O" : [
      (5 , 0),
       ( 3 , 2.277078e2),
       ( 1 , 3.265689e2),
       ( 5 , 2.283029e4),
       ( 1 , 4.861993e4)
    ],
    "N2": [
        (1,  0),
        (3 , 7.223157e4),
        (6 , 8.577863e4),
        (6 , 8.605027e4),
        (3 , 9.535119e4),
        (1 , 9.805636e4),
        (2 , 9.968268e4),
        (   2 , 1.048976e5),
        ( 5 , 1.116490e5),
        (   1 , 1.225836e5),
        (6,  1.248857e5),
        (6 , 1.282476e5),
        (10, 1.338061e5),
        (6 , 1.404296e5),
        (6 , 1.504959e5)
    ],
    "O2": [
        ( 3,  0),
        (2 , 1.139156e4),
        (1 , 1.898474e4),
        (1 , 4.755974e4),
        (6 , 4.991242e4),
        (3 , 5.092269e4),
        (3 , 7.189863e4)
    ],
    "NO": [
        (4 , 0),
        (8 , 5.467346e4),
        (2 , 6.317140e4),
        (4 , 6.599450e4),
        (4 , 6.906121e4),
        (4 , 7.049998e4),
        (4 , 7.491055e4),
        (2 , 7.628875e4),
        (4 , 8.676189e4),
        (2 , 8.714431e4),
        (4 , 8.886077e4),
        (4 , 8.981756e4),
        (2 , 8.988446e4),
        (2 , 9.042702e4),
        (2 , 9.064284e4),
        (4 , 9.111763e4)
    ],
}

In [20]:
ThetavsDict = {
    "N" : 0,
    "O" : 0,
    "N2": 3371,
    "O2": 2256,
    "NO": 2719
}

In [21]:
# Write a python class of species and mixture of gas components with possibilities to compute viscosity using sutherland model and mixing rule
# Add thermal conductivity eucken calculation and possibility of two temperature approach
# can you make it faster using numpy and njit
# all constants for different species properties should be stored in dicts, so we can invoke it as GasComponent['O2']. 

In [22]:
## Сначала весь функционал, потом оптимизация!

In [23]:
components = {
    'O2': {
        'molecular_weight': molecular_weights['O2'],
        'sutherland_constants': sutherland_constants['O2'],
    },
    'N2': {
        'molecular_weight': molecular_weights['N2'],
        'sutherland_constants': sutherland_constants['N2'],
    }
}

In [24]:
sutherland_constants['O2']

{'As': 1.458e-06, 'Ts': 120.0}

In [25]:
n = 25
mixture.iloc[n,:]

Temperature[K]                                    15447.08989
 Pressure[Pa]                                   198069.470344
 N                                                   0.203671
 O                                                   0.168494
 N2                                                  0.130765
 O2                                                  0.298519
 NO                                                  0.198552
 Specific heat at constant pressure [J/kg/K]      1580.006634
 Thermal conductivity translational [W/m/K]          0.428985
 Thermal conductivity internal [W/m/K]                0.34166
 Shear viscosity [Pa*s}                              0.000329
Unnamed: 11                                                  
Name: 25, dtype: object

In [26]:
T1, N1, O1, N21, O21, NO1 = mixture.iloc[n,0], mixture.iloc[n,2], mixture.iloc[n,3], mixture.iloc[n,4], mixture.iloc[n,5], mixture.iloc[n,6]

In [27]:
O2data.iloc[28087]

Temperature[K]                                  15448.016913
 Pressure[Pa]                                   76026.656242
 O2                                                        1
 Specific heat at constant pressure [J/kg/K]     1083.088567
 Thermal conductivity translational [W/m/K]         0.316931
 Thermal conductivity internal [W/m/K]              0.197738
 Shear viscosity [Pa*s}                             0.000325
Unnamed: 7                                                  
Name: 28087, dtype: object

In [28]:
R_universal = 8.31446261815324

In [29]:
### TODO: rewrite in more concise way, test it (write test), get into a lib

In [30]:
from numba import njit
import numpy as np

In [55]:
class Specie:
    def __init__(self, name, viscosity_model = "blottner", thermal_conductivity_model = "eucken"):
        self.name = name
        self.is_atom = species_type[name]
        self.molecular_weight = molecular_weights[name]
        self.sutherland_constant = sutherland_constants[name]
        self.blottner_constant = blottner_constants[name]
        self.viscosity_model = viscosity_model
        self.thermal_conductivity_model = thermal_conductivity_model
        self.R_specific = R_universal/self.molecular_weight*1000
        
    def __repr__(self):
        info =f"{self.name}"
        if self.is_atom:
            info += ", atom"
        else:
            info += ", molecule" #diatomic?
        info+= f", {self.viscosity_model} viscosity model"
        return info

    
    def compute_viscosity(self, T, p = None):
        if self.viscosity_model == "blottner":
            return self._compute_viscosity_blottner(T, self.blottner_constant['A'], 
                                                    self.blottner_constant['B'], 
                                                    self.blottner_constant['C'])
        elif self.viscosity_model == "sutherland":
            return self._compute_viscosity_sutherland(T, self.blottner_constant['As'], 
                                                      self.blottner_constant['Ts']
                                                     )
        else:
            raise ValueError("Invalid viscosity model specified.")
    
    def compute_thermal_conductivity(self, T, Tve = None):
        if Tve == None:
            Tve = T
            
        if self.thermal_conductivity_model == "eucken":
            return self._compute_thermal_conductivity_eucken(T, Tve, 
                                                             self.compute_viscosity(T), 
                                                             self._compute_C_v_tr(), 
                                                             self._compute_C_v_ve(Tve))
        return 0
        # two temperatures!
        
    @staticmethod
    def _compute_viscosity_sutherland(T, As, Ts):
        ''' \mu = A_s\frac{T^{\frac{3}{2}}}{T+T_s} '''
        return As*T**(1.5)/(Ts+T)
    
    @staticmethod
    def _compute_viscosity_blottner(T, As, Bs, Cs):
        '''  \mu=0.1\exp{[(A_s\ln{T}+B_s)\ln(T)+C_s]} '''
        return 0.1*np.exp((As*np.log(T)+Bs)*np.log(T)+Cs)
    
    @staticmethod
    def _compute_thermal_conductivity_eucken(T, Tve, mu, C_v_tr, C_v_ve):
        return (2.5*C_v_tr*mu, 1.25*C_v_ve*mu)

    @staticmethod
    def _compute_C_v_t(R_specific):
        return 1.5 * R_specific

    @staticmethod
    def _compute_C_v_r(R_specific):
        return R_specific

    def _compute_C_v_tr(self): #precompute it?
        return self._compute_C_v_t(self.R_specific) + self._compute_C_v_r(self.R_specific)
    
    
    @staticmethod
    def _compute_C_v_v(Tve, R_specific, thetavs, is_atom):
        if is_atom:
            return 0
        return R_specific * (thetavs/Tve)**2 * np.exp(thetavs/Tve) / (np.exp(thetavs/Tve)-1)**2
    
    @staticmethod
    def _compute_C_v_el(R_specific, Tve, gistheta):
        num1, den1, num21, num22 = 0.0, 0.0, 0.0, 0.0
        for i in range(len(gistheta)):
            E = np.exp(-gistheta[i][1]/Tve)
            num1 += gistheta[i][0]*E*(gistheta[i][1]/Tve)**2
            den1 += gistheta[i][0]*E
            num21 += gistheta[i][0]*gistheta[i][1]*E
            num22 += gistheta[i][0]*E*gistheta[i][1]/Tve**2

        return R_specific *(num1 - (num21*num22/den1)) / (den1)

    def _compute_C_v_ve (self, Tve):

        return self._compute_C_v_v(Tve, self.R_specific, ThetavsDict[self.name], self.is_atom) + \
               self._compute_C_v_el(Tve, self.R_specific, GisThetaDict[self.name])

    def compute_specific_heat_capacity_p (self):
        return self._compute_C_v_tr() + self.R_specific

In [70]:
class Mixture:
        def __init__(self, components, mole_fractions, mixing_rule = "wilke"):
            self.components = components
            self.num_components = len(mole_fractions)
            self.mole_fractions = mole_fractions
            self.mixing_rule = mixing_rule
            
        def __repr__(self):
            info = "Gas Mixture\n=======================\nComponents:\n"
            for comp, mole_fraction in zip(self.components, self.mole_fractions):
                info += f"- {comp.name}: Mole Fraction = {mole_fraction:.2f}\n"
            info+="=======================\n"+f"Current mixing rule: {self.mixing_rule}"+"\n======================="
            return info
          
        def compute_viscosity(self, T, p = None):
            species_viscosities, molecular_weights = np.empty(self.num_components), np.empty(self.num_components)
            for i in range(self.num_components):
                species_viscosities[i] = self.components[i].compute_viscosity(T,p)
                molecular_weights[i] = self.components[i].molecular_weight 
                
            if self.mixing_rule == "wilke":
                return self._compute_wilke_mixing_rule(self.num_components, species_viscosities, self.mole_fractions, molecular_weights)
            
        def compute_thermal_conductivity(self, T, Tve = None):
            if Tve == None:
                Tve = T
            species_thermal_conductivity_tr = np.empty(self.num_components)
            species_thermal_conductivity_ve =  np.empty(self.num_components)
            molecular_weights = np.empty(self.num_components)
            for i in range(self.num_components):
                species_thermal_conductivity_tr[i], species_thermal_conductivity_ve[i] = self.components[i].compute_thermal_conductivity(T,Tve)
                molecular_weights[i] = self.components[i].molecular_weight 
            print(species_thermal_conductivity_ve)
                
            if self.mixing_rule == "wilke":
                thermal_conductivity_tr = self._compute_wilke_mixing_rule(self.num_components, species_thermal_conductivity_tr, self.mole_fractions, molecular_weights)
                thermal_conductivity_ve = self._compute_wilke_mixing_rule(self.num_components, species_thermal_conductivity_ve, self.mole_fractions, molecular_weights)
                return thermal_conductivity_tr, thermal_conductivity_ve

            
        @staticmethod    
        def _compute_wilke_mixing_rule(num_components, species_properties, mole_fractions, molecular_weights):
            result = 0.0

            for j in range(num_components):
                phi_j = 0.0
                for k in range(num_components):
                    phi_j += mole_fractions[k] * ((1.0 + np.sqrt(species_properties[j] / species_properties[k]) * \
                            (molecular_weights[k] / molecular_weights[j]) ** 0.25) ** 2.0) / \
                            np.sqrt(8.0 * (1.0 + molecular_weights[j] / molecular_weights[k]))

                result += mole_fractions[j] * species_properties[j] / phi_j

            return result

In [71]:
O2, N2, N, O, NO = Specie("O2"), Specie("N2"), Specie("N"), Specie("O"), Specie("NO")

In [72]:
air = Mixture([O2,N2, NO, N, O],[O21,N21, NO1, N1, O1])

In [73]:
air.compute_viscosity(T1)

0.00032334902060595516

In [74]:
O2.compute_specific_heat_capacity_p()

909.4284524274767

In [75]:
O2.compute_thermal_conductivity(T1)

(0.482380065036104, 0.09630471199836926)

In [76]:
O.compute_thermal_conductivity(T1)

(1.2360819099437745, 0.3883998643364477)

In [77]:
O.compute_viscosity(T1)

0.0003805624081395269

In [78]:
air.compute_thermal_conductivity(T1,T1/2)

[9.57929943e-02 1.15534806e-01 9.96437142e-02 1.18439190e-16
 1.94199932e-01]


(0.654922501291253, 3.208638964421172e-15)

In [24]:
mixture.iloc[n,:]

Temperature[K]                                    15447.08989
 Pressure[Pa]                                   198069.470344
 N                                                   0.203671
 O                                                   0.168494
 N2                                                  0.130765
 O2                                                  0.298519
 NO                                                  0.198552
 Specific heat at constant pressure [J/kg/K]      1580.006634
 Thermal conductivity translational [W/m/K]          0.428985
 Thermal conductivity internal [W/m/K]                0.34166
 Shear viscosity [Pa*s}                              0.000329
Unnamed: 11                                                  
Name: 25, dtype: object

In [25]:
air.components

[O2, molecule, blottner viscosity model,
 N2, molecule, blottner viscosity model,
 NO, molecule, blottner viscosity model,
 N, atom, blottner viscosity model,
 O, atom, blottner viscosity model]

In [45]:
air.compute_viscosity(T1)

0.00032334902060595516

### Определяем константы

#### константы моделей сазерленда, блоттнера

### Определяем необходимые функции

#### формула смешивания Вилке

#### Модели сазерленда, блотнера
 